## Imports

In [0]:
# https://medium.com/@move37timm/using-kaggle-api-for-google-colaboratory-d18645f93648
from googleapiclient.discovery import build
import io, os
from googleapiclient.http import MediaIoBaseDownload
from google.colab import auth, files

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

## Get Data

In [108]:
# Get data from kaggle
# https://medium.com/@move37timm/using-kaggle-api-for-google-colaboratory-d18645f93648
!pip install kaggle

# Copy + pasted from medium article:

auth.authenticate_user()

drive_service = build('drive', 'v3')
results = drive_service.files().list(
        q="name = 'kaggle.json'", fields="files(id)").execute()
kaggle_api_key = results.get('files', [])

# Had to change this from /content to /root.
filename = "/root/.kaggle/kaggle.json"
os.makedirs(os.path.dirname(filename), exist_ok=True)

request = drive_service.files().get_media(fileId=kaggle_api_key[0]['id'])
fh = io.FileIO(filename, 'wb')
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print("Download %d%%." % int(status.progress() * 100))
os.chmod(filename, 600)

# API call from Kaggle
!kaggle competitions download -c wine-quality-linear-regression-2


Download 100%.
winequality-white-sample.csv: Skipping, found more recently modified local copy (use --force to force download)
winequality-white-testing.csv: Skipping, found more recently modified local copy (use --force to force download)
winequality-white-training.csv: Skipping, found more recently modified local copy (use --force to force download)


## Read, Scale, and Split Data

In [109]:
SEED=23409
# Grab data.
df_train = pd.read_csv('/content/winequality-white-training.csv', index_col=0)
df_test = pd.read_csv('/content/winequality-white-testing.csv', index_col=0)
df_sample = pd.read_csv('/content/winequality-white-sample.csv', index_col=0)

print(df_train.shape)
print(df_test.shape)
print(df_train.head(10))
print(df_train.describe())

#for f in df_train.columns.drop('quality'):
#  fig, ax = plt.subplots(1, 1)
#  ax.boxplot(df_train[f], whis=[1, 99])
#  ax.set_title(f)

# Extract features and labels
x = df_train.drop('quality', axis=1).values
y = df_train['quality'].values

# Scale data
scaler = StandardScaler()
scaler.fit(x)
x_scaled = scaler.transform(x)

# Split data.
x_train, x_test, y_train, y_test = train_test_split(x_scaled, y, test_size=0.25,
                                                    random_state=SEED)

(3898, 12)
(1000, 11)
    fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
Id                                                                            
0             8.7              0.23         0.32           13.40      0.044   
1             5.0              0.27         0.40            1.20      0.076   
2             8.0              0.16         0.36            1.50      0.033   
3             5.8              0.28         0.35            2.30      0.053   
4             7.1              0.14         0.33            1.00      0.104   
5             7.5              0.30         0.21            6.55      0.026   
6             6.6              0.41         0.27           10.70      0.110   
7             6.2              0.30         0.32            1.20      0.052   
8             6.8              0.23         0.32            1.60      0.026   
9             6.8              0.22         0.31            1.40      0.053   

    free sulfur dioxide  tota

## Train and Test Network

In [110]:
# model = keras.Sequential([
#   keras.layers.Dense(x.shape[1], input_shape=(x.shape[1],)),
#   keras.layers.PReLU(),
#   keras.layers.Dense(64),
#   keras.layers.PReLU(),
#   keras.layers.Dense(32),
#   keras.layers.PReLU(),
#   #keras.layers.Dense(x.shape[1], activation=tf.nn.relu),
#   keras.layers.Dense(1)
# ])

model = keras.Sequential([
  #keras.layers.Dense(x.shape[1], input_shape=(x.shape[1],)), # Linear activation
  keras.layers.Dense(8, activation=tf.nn.relu, input_shape=(x.shape[1],)),
  keras.layers.Dense(4, activation=tf.nn.relu),
  #keras.layers.Dense(x.shape[1], activation=tf.nn.relu),
  keras.layers.Dense(1)
])

optimizer = tf.train.RMSPropOptimizer(0.001)
model.compile(loss='mse', optimizer=optimizer, metrics=['mse'])
early_stop = keras.callbacks.EarlyStopping(monitor='val_mean_squared_error',
                                           patience=7)

model.fit(x_train, y_train, epochs=200, callbacks=[early_stop],
          validation_data=(x_test, y_test))

Train on 2923 samples, validate on 975 samples
Epoch 1/200
2923/2923 [==============================] - 4s 1ms/step - loss: 35.2647 - mean_squared_error: 35.2647 - val_loss: 30.4623 - val_mean_squared_error: 30.4623
Epoch 2/200
2923/2923 [==============================] - 1s 256us/step - loss: 26.8649 - mean_squared_error: 26.8649 - val_loss: 22.1932 - val_mean_squared_error: 22.1932
Epoch 3/200
2923/2923 [==============================] - 1s 238us/step - loss: 18.4480 - mean_squared_error: 18.4480 - val_loss: 14.0245 - val_mean_squared_error: 14.0245
Epoch 4/200
2923/2923 [==============================] - 1s 240us/step - loss: 10.6449 - mean_squared_error: 10.6449 - val_loss: 7.3360 - val_mean_squared_error: 7.3360
Epoch 5/200
2923/2923 [==============================] - 1s 270us/step - loss: 5.4446 - mean_squared_error: 5.4446 - val_loss: 4.1463 - val_mean_squared_error: 4.1463
Epoch 6/200
2923/2923 [==============================] - 1s 307us/step - loss: 3.4473 - mean_squared_error

In [111]:
# 0.52: Two dense layers, 64 neurons each (0.71 Kaggle score)
# 0.45: Add input layer, two dense, 64 neurons each (0.77 Kaggle score)
# 0.48: Same as above, reduced to 32 neurons per layer
# 0.45: Input layer, 32, 16
# 0.47: Input layer, 32
# 0.49: Input, 32, 16, 11
# 0.49: Input layer, 32, 16, upped patience to 5
# 0.48: Changed activations to PReLU, input, 32, 16, 1 (Kaggle score 0.754)
# 0.54: PReLU, input, 64, 32, 1 (maybe worth submitting, wine_054)
# 0.58: PReLU, input (linear), 64, 32, 1
# 0.51: relu, input (linear), 64, 32, 1 (maybe worth submitting, wine_051)
# 0.56: relu, input (linear), 32, 32, 1
# 0.47: relu, no linear input, 32, 16, 1 (maybe worth submitting, wine_047)
# 0.48: relu, no linear input, 16, 8, 1 (maybe worth submitting, wine_048)
# 0.47: relu, no linear input, 8, 4, 1 (maybe worth submitting, wine_047_2)
# 0.49: relu, no linear input, 8, 4, 1, patience for retrain @ 3 (wine_049)
#       scored 0.75397 on Kaggle
[loss, mse] = model.evaluate(x_test, y_test, verbose=0)

print("Testing set Mean Squared Error: {:.2f}".format(mse))

Testing set Mean Squared Error: 0.49


## Retrain, Predict, Write Output

In [112]:
early_stop = keras.callbacks.EarlyStopping(monitor='mean_squared_error',
                                           patience=3)
model.fit(x_scaled, y, epochs=200, callbacks=[early_stop])
y_test = model.predict(scaler.transform(df_test.values))
# print(y_test[0:5,:])
df_sample['quality'] = y_test
df_sample.to_csv('wine.csv')
files.download('wine.csv')

Epoch 1/200
3898/3898 [==============================] - 1s 238us/step - loss: 0.5043 - mean_squared_error: 0.5043
Epoch 2/200
3898/3898 [==============================] - 1s 244us/step - loss: 0.5019 - mean_squared_error: 0.5019
Epoch 3/200
3898/3898 [==============================] - 1s 197us/step - loss: 0.5011 - mean_squared_error: 0.5011
Epoch 4/200
3898/3898 [==============================] - 1s 216us/step - loss: 0.5000 - mean_squared_error: 0.5000
Epoch 5/200
3898/3898 [==============================] - 1s 223us/step - loss: 0.5001 - mean_squared_error: 0.5001
Epoch 6/200
3898/3898 [==============================] - 1s 261us/step - loss: 0.4997 - mean_squared_error: 0.4997
Epoch 7/200
3898/3898 [==============================] - 1s 159us/step - loss: 0.4987 - mean_squared_error: 0.4987
Epoch 8/200
3898/3898 [==============================] - 1s 213us/step - loss: 0.4969 - mean_squared_error: 0.4969
Epoch 9/200
3898/3898 [==============================] - 1s 216us/step - loss: 0